In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%cd ../../

C:\Projects\python\recommender


In [3]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp

In [4]:
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHA_PATH = Path("./inputs/topcoder/challenge_sm.csv")
regs_min = 4
cha_min = 4

In [5]:
regs_df = pd.read_csv(REGS_PATH)
regs_df.head()

,challengeId,registant,date
0,30044052,phead,2014-07-09
1,30044052,lovefreya,2014-07-09
2,30044052,TMALBONPH,2014-07-09
3,30044052,daga_sumit,2014-07-09
4,30044052,abcivashritt,2014-07-09


## Add Challenge DataFrame

In [6]:
from ast import literal_eval

In [7]:
attr_df = pd.read_csv("./inputs/topcoder/challenge.csv",
                      infer_datetime_format=True,
                      converters={
                          'technologies': literal_eval,
                          'platforms': literal_eval
                      })
attr_df.head()

,challengeId,date,prizes,technologies,platforms
0,30044052,2014-07-09,1800.0,[HTML],[HTML]
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]"
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS]
3,30044055,2014-07-09,400.0,[iOS],[iOS]
4,30044056,2014-07-09,400.0,[iOS],[iOS]


In [8]:
chag_df = regs_df[['challengeId']]
chag_df.shape

(610025, 1)

In [9]:
chag_df = chag_df.drop_duplicates(subset=['challengeId'])
chag_df.shape

(39916, 1)

In [10]:
chag_df = pd.merge(left=chag_df, right=attr_df, how='inner', on='challengeId')
chag_df.head()

,challengeId,date,prizes,technologies,platforms
0,30044052,2014-07-09,1800.0,[HTML],[HTML]
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]"
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS]
3,30044055,2014-07-09,400.0,[iOS],[iOS]
4,30044056,2014-07-09,400.0,[iOS],[iOS]


In [11]:
chag_df['date'] = pd.to_datetime(chag_df['date'])
chag_df.date.dtype

dtype('<M8[ns]')

In [12]:
chag_df['per'] = chag_df.date.dt.to_period("M")
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
0,30044052,2014-07-09,1800.0,[HTML],[HTML],2014-07
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]",2014-07
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS],2014-07
3,30044055,2014-07-09,400.0,[iOS],[iOS],2014-07
4,30044056,2014-07-09,400.0,[iOS],[iOS],2014-07


In [13]:
pd.Period(value='2005-1')

Period('2010-01', 'M')

In [14]:
pd.Timestamp('2005-01-01 00:00:00')

Timestamp('2010-01-01 00:00:00')

In [15]:
chag_df['date'][0]

Timestamp('2014-07-09 00:00:00')

## Encode for period

In [16]:
from sklearn.preprocessing import OrdinalEncoder

In [17]:
chag_df = chag_df.sort_values(by=['per'])
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
31190,30000000,2006-10-20,0.0,[],[],2006-10
31189,30000002,2006-10-20,0.0,[],[],2006-10
31188,30000018,2006-10-30,0.0,[],[],2006-10
31187,30000030,2006-11-04,0.0,"[Java, HTTP, JSF]",[],2006-11
31095,30000196,2006-11-23,0.0,[Java],[],2006-11


In [18]:
period_encoder = OrdinalEncoder(categories='auto')
period_encoder.fit(chag_df[['per']])

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [19]:
batch_df = chag_df.sample(n=3)
batch_df.head()

,challengeId,date,prizes,technologies,platforms,per
23944,30033124,2013-03-09,1800.0,"[C++, HTTP, JavaScript, C, Objective C]",[],2013-03
38021,30003203,2008-01-03,500.0,"[.NET, Windows Communication Foundation]",[],2008-01
9384,30057490,2017-04-28,1200.0,[Java],[Other],2017-04


In [23]:
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
31190,30000000,2006-10-20,0.0,[],[],2006-10
31189,30000002,2006-10-20,0.0,[],[],2006-10
31188,30000018,2006-10-30,0.0,[],[],2006-10
31187,30000030,2006-11-04,0.0,"[Java, HTTP, JSF]",[],2006-11
31095,30000196,2006-11-23,0.0,[Java],[],2006-11


In [22]:
period_encoder.transform(chag_df.head()[['per']])

array([[0.],
       [0.],
       [0.],
       [1.],
       [1.]])

## Encode for technologies

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

In [19]:
tech_bin = MultiLabelBinarizer(sparse_output=True)
tech_bin.fit(chag_df['technologies'])

MultiLabelBinarizer(classes=None, sparse_output=True)

In [22]:
batch_df = chag_df.sample(n=10)
batch_df.head()

,challengeId,date,prizes,technologies,platforms,per
25895,30029170,2012-08-02,500.0,[],[],2012-08
13594,30062575,2018-01-31,50.0,[],[],2018-01
8867,30056827,2017-03-15,1300.0,"[ReactJS, Node.js]",[Heroku],2017-03
16192,30066470,2018-06-15,50.0,[Other],[Other],2018-06
36373,30005600,2008-12-02,1000.0,"[J2EE, Java, HTML, EJB 3, JPA, Hibernate, MySQL]",[],2008-12


In [24]:
tech_bin.transform(batch_df['technologies'].repeat(2))

<20x216 sparse matrix of type '<class 'numpy.int32'>'
	with 42 stored elements in Compressed Sparse Row format>

## Preprocess Registers Dataframe

In [6]:
regs_df = pd.read_csv(REGS_PATH)
regs_df.head()

,challengeId,registant,date
0,30044052,phead,2014-07-09
1,30044052,lovefreya,2014-07-09
2,30044052,TMALBONPH,2014-07-09
3,30044052,daga_sumit,2014-07-09
4,30044052,abcivashritt,2014-07-09


In [72]:
regs_df['date'] = pd.to_datetime(regs_df['date'])
regs_df.dtypes

challengeId             int64
registant              object
date           datetime64[ns]
dtype: object

In [73]:
regs_counts = regs_df['registant'].value_counts()
regs_counts.head()

sunbinbrother    7451
poundinc_tc      6992
woodjhon         6398
luckvin          6039
Schpotsky        5757
Name: registant, dtype: int64

In [74]:
regs_counts.shape

(60017,)

In [75]:
regs_counts = regs_counts[regs_counts >= 4]
regs_counts.shape

(9894,)

In [76]:
regs_df.shape

(610025, 3)

In [77]:
regs_df = regs_df[regs_df['registant'].isin(regs_counts.index)]
regs_df.shape

(544568, 3)

In [78]:
regs_df['period'] = regs_df['date'].dt.to_period("M")
regs_df.head()

,challengeId,registant,date,period
0,30044052,phead,2014-07-09,2014-07
1,30044052,lovefreya,2014-07-09,2014-07
2,30044052,TMALBONPH,2014-07-09,2014-07
3,30044052,daga_sumit,2014-07-09,2014-07
4,30044052,abcivashritt,2014-07-09,2014-07


In [82]:
regs_df = regs_df.sort_values(by=['registant', 'date'])
regs_df.head()

,challengeId,registant,date,period
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09
482618,30000143,-Neo-,2006-11-16,2006-11


In [83]:
regs_df['previousId'] = regs_df['challengeId']
regs_df['prevPeriod'] = regs_df['period']
regs_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,30022003,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08,30045145,2014-08
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09,30045639,2014-09
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09,30045678,2014-09
482618,30000143,-Neo-,2006-11-16,2006-11,30000143,2006-11


In [84]:
regs_df['previousId'] = regs_df['previousId'].shift(
    periods=1).fillna(0).astype('int64')
regs_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,0,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08,30022003,2014-08
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09,30045145,2014-09
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09,30045639,2014-09
482618,30000143,-Neo-,2006-11-16,2006-11,30045678,2006-11


In [85]:
fill_v = regs_df['period'].iloc[0]
fill_v

Period('2010-03', 'M')

In [86]:
regs_df['prevPeriod'] = regs_df['prevPeriod'].shift(periods=1).fillna(fill_v)
regs_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,0,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08,30022003,2010-03
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09,30045145,2014-08
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09,30045639,2014-09
482618,30000143,-Neo-,2006-11-16,2006-11,30045678,2014-09


In [87]:
first_mask = regs_df.duplicated(subset=['registant'], keep='first')
first_mask.head()

446754    False
10253      True
16217      True
16674      True
482618    False
dtype: bool

In [88]:
regs_df['previousId'] = regs_df['previousId'].where(first_mask, -1)
regs_df[~first_mask].head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,-1,2010-03
482618,30000143,-Neo-,2006-11-16,2006-11,-1,2014-09
166877,30055022,-_-33,2016-08-19,2016-08,-1,2007-08
588599,30003421,-jacob-,2008-01-24,2008-01,-1,2016-12
588099,30003490,-kotenok-,2008-01-31,2008-01,-1,2014-07


## Batch collate function

In [89]:
batch_df = regs_df.sample(n=10)
batch_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
519872,30011548,laura_p,2010-05-12,2010-05,30011545,2010-05
83677,30050748,alfiya_Zi,2015-07-09,2015-07,30050746,2015-07
572147,30005363,Veonax,2008-10-13,2008-10,30021002,2008-10
19759,30045980,sunbinbrother,2014-09-23,2014-09,30045979,2014-09
479623,30017204,LOY,2011-04-19,2011-04,30015818,2010-11


In [94]:
# Sampel negative df list
per_series = batch_df.period
per_series

519872    2010-05
83677     2015-07
572147    2008-10
19759     2014-09
479623    2011-04
609627    2006-12
179195    2016-10
528394    2010-03
563840    2009-02
450520    2009-09
Name: period, dtype: period[M]

In [92]:
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
0,30044052,2014-07-09,1800.0,[HTML],[HTML],2014-07
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]",2014-07
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS],2014-07
3,30044055,2014-07-09,400.0,[iOS],[iOS],2014-07
4,30044056,2014-07-09,400.0,[iOS],[iOS],2014-07


In [97]:
neg_list = [chag_df[chag_df['per'] == per].sample(n=2) for per in per_series]
neg_list[0]

,challengeId,date,prizes,technologies,platforms,per
29164,30022082,2010-05-21,750.0,[],[],2010-05
33765,30011050,2010-05-04,800.0,"[C#, .NET 3.5]",[],2010-05


In [98]:
pd.concat(neg_list)

,challengeId,date,prizes,technologies,platforms,per
29164,30022082,2010-05-21,750.0,[],[],2010-05
33765,30011050,2010-05-04,800.0,"[C#, .NET 3.5]",[],2010-05
4982,30050970,2015-07-30,1.0,[],[],2015-07
4836,30050747,2015-07-09,1300.0,[],[],2015-07
36571,30005312,2008-10-01,1250.0,"[.NET, C#, Windows Communication Foundation]",[],2008-10
36500,30005422,2008-10-23,100.0,[Flex],[],2008-10
1146,30045542,2014-09-02,250.0,[HTML5],[HTML],2014-09
1175,30045578,2014-09-03,250.0,"[PostgreSQL, Node.js, Angular.js (1.0)]",[NodeJS],2014-09
31237,30016793,2011-04-04,500.0,"[J2EE, Java, Spring, Hibernate, IBM DB2]",[],2011-04
30933,30017209,2011-04-18,300.0,[],[],2011-04


In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
chag_encoder = OneHotEncoder(categories='auto', handle_unknown='error')
chag_encoder.fit(regs_df[['challengeId']])

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [9]:
batch_df = regs_df.sample(n=5)
batch_df.head()

,challengeId,registant,date
109963,30052023,naeem95m,2016-01-16
274846,30067807,feiyusk,2018-07-12
100732,30051596,m-v-kachalov,2015-10-02
423947,30025000,caru,2011-12-12
355629,30034436,lanchongyizu,2014-01-29


In [22]:
chag_vector = chag_encoder.transform(batch_df[['challengeId']])
chag_vector

<5x39916 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [23]:
chag_vector.data

array([1., 1., 1., 1., 1.])

In [19]:
chag_vector = sp.hstack([chag_vector, chag_vector])
chag_vector.shape

(5, 79832)

In [20]:
chag_vector

<5x79832 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in COOrdinate format>

In [25]:
cols = chag_vector.indices
cols

array([27291, 38361, 26998, 12074, 16488])

In [31]:
batch_df[['challengeId']].values.repeat(2, axis=0)

array([[30052023],
       [30052023],
       [30067807],
       [30067807],
       [30051596],
       [30051596],
       [30025000],
       [30025000],
       [30034436],
       [30034436]], dtype=int64)

In [34]:
batch_df['registant'].repeat(2)

109963        naeem95m
109963        naeem95m
274846         feiyusk
274846         feiyusk
100732    m-v-kachalov
100732    m-v-kachalov
423947            caru
423947            caru
355629    lanchongyizu
355629    lanchongyizu
Name: registant, dtype: object